In [2]:
import torch
import numpy as np
import nibabel as ni
import os, shutil
import time
import random
import pandas as pd 
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
import loss
from dataset import MRI_dataloader
from model import VAE
from tqdm import tqdm

device = 'cuda'

ckpt_path = path2save = "./checkpoint/model_vae_epoch_{}.pt"
ckpt_path = ckpt_path.format(52)
path = 'dataset/images'
mri_loader = MRI_dataloader(path)
batch_size = 8
dataloader = DataLoader(mri_loader, batch_size=batch_size, shuffle=True)
model = VAE().to(device)

import pandas as pd

labels = pd.read_csv('dataset/data.csv')

# model.eval()
# for batch in tqdm(dataloader):
#     batch = batch.to(device)
#     y, z_mean, z_log_sigma = model(batch)
#     print(y.shape)

#     break

  0%|                                                                                                 | 0/22 [00:06<?, ?it/s]

torch.Size([8, 1, 137, 128, 128])


In [5]:
latent_dim = 512
epsilon = torch.normal(size=(1, latent_dim), mean=0, std=1.0, device=device)
z = z_mean + z_log_sigma.exp()*epsilon

In [6]:
z.shape

torch.Size([8, 512])

In [14]:
from scipy.ndimage import zoom
import nibabel as ni

def preprocess_image(path, filename, img_size=(137, 128, 128)):
    image = ni.load(os.path.join(path, filename)) #self.filenames[index]
    image = np.array(image.dataobj)
    image = np.moveaxis(image, [0, 1], [1, 0])
    img_scale = tuple([img_size[i]/image.shape[i] for i in range(len(image.shape))])
    image = zoom(image, img_scale)
    image = image / 255.
    image = torch.from_numpy(image)
    image = image.unsqueeze(0)
    #image = torch.permute(image, (1, 0, 2)).unsqueeze(0)
    return image

In [50]:
filenames = [i for i in os.listdir(path) if i.endswith(".nii")]
model.eval()
PT_500 = []
PT_4000 = []
latents = []
for filename in filenames:
    img = preprocess_image(path, filename)
    img = img.unsqueeze(0).to(device)
    y, z_mean, z_log_sigma = model(img)
    latent = z_mean + z_log_sigma.exp()*epsilon
    
    filename_ = filename.split('_')[0]
    fl = filename_[:5] + '_' + filename_[5:]
    PT_500.append(labels[labels['ID'] == fl]['PT500'][0])
    PT_4000.append(labels[labels['ID'] == fl]['PT4000'][0])
    latents.append(latent.detach().cpu().numpy())
    
    
    break

In [ ]:
# split into 70 30 for training and testing
# train model on 70 and test on 30%
# report rmse scores for different models
# add data augmentation and train again